<a href="https://colab.research.google.com/github/GoalKepper/NeiroCourse/blob/main/%D0%A5%D0%B0%D0%BA%D0%B0%D1%82%D0%BE%D0%BD_%D0%9C%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import nltk 
nltk.download('punkt') 
import numpy as np
import random 
import string
import re 

# !pip install pymorphy2
import pymorphy2

import heapq

import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
with open("Data.txt", "r") as read_file:
    text = json.load(read_file)

for i in range(10):
    print(text[i])

list_news = []
for i in range(len(text)):
    list_news.append(text[i]['text'])
list_news = list_news[:30]

print(list_news)
k = 0
for i in range(len(list_news)):
    k += len(list_news[i].split())

print(k)

{'id': 1, 'title': 'МЭР прогнозирует снижение числа плановых проверок бизнеса в 2023 году', 'text': 'Российские органы контроля всех уровней запланировали провести в 2023 году около 250 тыс. проверок бизнеса, что примерно на 110 тыс. меньше, чем было внесено в планы 2022 года. При этом количество надзорных мероприятий может еще сократиться после согласования плана с прокуратурой, сообщил замминистра экономического развития РФ Алексей Херсонцев, чьи слова приводит ТАСС. «Сейчас у нас органы контроля всех уровней направили заявки, то есть при планировании зафиксировали на следующий год порядка 250 тысяч плановых проверок, <...> это на 110 тысяч проверок меньше, чем они планировали в прошлом году сделать на этот год. Понятно, что сейчас еще прокуратуры будут эти планы смотреть, согласовывать, очевидно, где-то их прореживать. Ориентир такой, думаю, что он еще будет снижаться», — сказал Херсонцев. Подписаться на Банки.ру в Telegram По его словам, сокращение числа проверок связано с постанов

In [ ]:
def parse_sentence(parse_text: list) -> list:

    text = parse_text.copy()
    morph = pymorphy2.MorphAnalyzer()

    for i in range(len(text)):
        text[i] = text[i].lower()
        text[i] = re.sub(r'\W', ' ', text[i])
        text[i] = re.sub(r'\s+', ' ', text[i])
        text[i] = text[i].replace('\n', ' ')
        ex_sentence = ""
        for word in text[i].split():
            p = morph.parse(word)[0] 
            ex_sentence += p.normal_form + " "
        text[i] = ex_sentence

    return text

In [ ]:
def get_most_freq_words(parse_text: list, delete_values_count: int = 20) -> list:

    text = parse_sentence(parse_text)

    wordfreq = {}

    for sentence in text:
        tokens = nltk.word_tokenize(sentence)
        for token in tokens:
            try:
                int(token)
                continue
            except:
                if token not in wordfreq.keys():
                    wordfreq[token] = 1
                else:
                    wordfreq[token] += 1
    most_freq = heapq.nlargest(len(wordfreq), wordfreq, key=wordfreq.get)
    if (delete_values_count != 0):
        most_freq = most_freq[int(delete_values_count / 2):int(-delete_values_count / 2)]

    return most_freq

In [ ]:
def get_dict_words(parse_text: list, n: int = 2, delete_values_count: int = 20) -> dict:

    most_freq = get_most_freq_words(parse_text, delete_values_count)
    text = parse_text.copy()

    check_dict = {}
    check_dict['names'] = most_freq
    for word in most_freq:
        check_dict[word] = {}

    text = parse_sentence(text)

    for sentence in text:
        values = sentence.split()
        for i in range(len(values)):
            if values[i] in check_dict.keys():
                for j in range(max(0, i - n), min(i + n, len(values))):
                    if values[j] in check_dict.keys():
                        if values[i] not in check_dict[values[j]].keys():
                            check_dict[values[j]][values[i]] = 0
                        check_dict[values[j]][values[i]] += 1
                        
    return check_dict

In [ ]:
def check_dict_news(text1: str, text2: str, n: int = 2, most_freq_words_checked: int = 20, max_repetitions_count: int = 10, delete_values_count1: int = 20, delete_values_count2: int = 20) -> bool:
  
    most_freq1 = get_most_freq_words([text1], delete_values_count1)
    most_freq2 = get_most_freq_words([text2], delete_values_count2)

    dict1 = get_dict_words([text1], n, delete_values_count1)
    dict2 = get_dict_words([text2], n, delete_values_count2)

    repetitions_count = 0
    for word in most_freq1[:most_freq_words_checked]:
        if word in most_freq2[:most_freq_words_checked] and word != 'names':
            for word_in in dict1[word]:
                if word_in in dict2[word]:
                    repetitions_count += 1 + (dict1[word][word_in] - dict2[word][word_in]) / max(dict1[word][word_in], dict2[word][word_in])
                    if repetitions_count >= max_repetitions_count:
                        return True

    return False

In [ ]:
def get_all_news(text, max_cnts: int = 100, most_freq_words_checked: int = 20, max_repetitions_count: int = 10, delete_values_count: int = 20) -> list:

    list_news_final = []
    cnts = 0
    for sentence in text:
        for check_sentence in list_news_final:
            if check_dict_news(sentence, check_sentence, 20, most_freq_words_checked, max_repetitions_count, delete_values_count, delete_values_count):
                break
        else:
            list_news_final.append(sentence)
            cnts += 1
            if cnts >= max_cnts:
                break

    return list_news_final

In [ ]:
def find_news(text, need_text, max_cnts: int = 100, most_freq_words_checked: int = 20, max_repetitions_count: int = 10, delete_values_count1: int = 20, delete_values_count2: int = 5) -> list:

    all_news = get_all_news(text, max_cnts, most_freq_words_checked, max_repetitions_count, delete_values_count1)

    finded_news = []
    for sentence in all_news:
        if check_dict_news(sentence, need_text, 20, most_freq_words_checked, max_repetitions_count, delete_values_count1=delete_values_count1, delete_values_count2=delete_values_count2):
            finded_news.append(sentence)

    return finded_news

In [ ]:
read_file = open('Model.txt', 'rb')
model = pickle.load(read_file)
read_file.close()

In [ ]:
def get_trends(text, max_cnts: int = 100, most_freq_words_checked: int = 20, max_repetitions_count: int = 10, need_popular_words_count: int = 5, popular_level: int = 10) -> list:

    all_news = get_all_news(text, max_cnts, most_freq_words_checked, max_repetitions_count, 20)

    text_count = 0
    trends = []
    for sentence in all_news:
        sentence_count = 0
        for word in sentence.split():
            if word in model.keys():
                if model[word][word] >= popular_level:
                    sentence_count += 1
                    if sentence_count >= need_popular_words_count:
                        trends.append(sentence)
                        text_count += 1
                        if text_count >= max_cnts:
                            break
    
    return trends[max_cnts:]

In [ ]:
need = """Бухгалтер (часто отдельное подразделение) –
отвечает за операционную поддержку бизнеса,
регулярную отчётность и выполнение требований
законодательства. Этой роли важно быть в курсе
последней информации о нормативных актах,
условиях работы с банками и взаимодействия с
контрагентами"""

print(*find_news(list_news, need, max_cnts=2, most_freq_words_checked=100000, max_repetitions_count=3, delete_values_count1=10, delete_values_count2=0), sep='\n')

Российские органы контроля всех уровней запланировали провести в 2023 году около 250 тыс. проверок бизнеса, что примерно на 110 тыс. меньше, чем было внесено в планы 2022 года. При этом количество надзорных мероприятий может еще сократиться после согласования плана с прокуратурой, сообщил замминистра экономического развития РФ Алексей Херсонцев, чьи слова приводит ТАСС. «Сейчас у нас органы контроля всех уровней направили заявки, то есть при планировании зафиксировали на следующий год порядка 250 тысяч плановых проверок, <...> это на 110 тысяч проверок меньше, чем они планировали в прошлом году сделать на этот год. Понятно, что сейчас еще прокуратуры будут эти планы смотреть, согласовывать, очевидно, где-то их прореживать. Ориентир такой, думаю, что он еще будет снижаться», — сказал Херсонцев. Подписаться на Банки.ру в Telegram По его словам, сокращение числа проверок связано с постановлением правительства РФ, которым установлено, что плановые проверки в 2023 году коснутся только предп

In [ ]:
print(*get_trends(list_news, max_cnts=4), sep='\n')

Второй по величине банк Швейцарии Credit Suisse намерен выкупить собственные облигации на общую сумму около 3 млрд долларов, сообщило агентство Bloomberg. Кредитная организация планирует выкупить ценные бумаги в евро и фунтах стерлингов на сумму порядка 975 млрд долларов, а также ценные бумаги в долларах на сумму до 2 млрд долларов, уточняет издание. Подписаться на Банки.ру в Telegram Банк также намерен приобрести ценные бумаги по значительно сниженным ценам. Например, Credit Suisse заплатит менее 96 центов за 1 евро, чтобы приобрести ценную бумагу с плавающей процентной ставкой на 750 млн евро, отмечает Bloomberg.
Российские индексы снизились по итогам основных фондовых торгов в четверг, а ключевые валюты подорожали к рублю на Московской бирже. Индекс Мосбиржи понизился на 3,76% — до отметки 1 944,75 пункта, а долларовый индекс РТС просел на 3,81% — до уровня 1 005,04 пункта. В отраслевом разрезе хуже рынка выглядели финансовый и нефтегазовый секторы, а потери меньше рыночных понес тр